In [8]:
import time
import logging
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from bs4 import BeautifulSoup
import os

In [3]:
# configure logging
logging.basicConfig(filename='error.log', level=logging.ERROR)

chrome_path = 'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe'
df = pd.read_excel(r'C:\Users\U6060174\OneDrive - Clarivate Analytics\Desktop\Files\Jupyter Notebook Work\Excel File\url.xlsx')

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.binary_location = chrome_path

driver = webdriver.Chrome(options=chrome_options)
retry_count = 3  # number of times to retry if an error occurs

wait = WebDriverWait(driver, 30)

for i, row in df.iterrows():
    url = row['{URL}']
    name = row['{Name}']
    assignments_name = row['{Assignments}']
    for _ in range(retry_count):  # retry loop
        try:
            driver.delete_all_cookies()
            driver.set_page_load_timeout(30)  # set a page load timeout of 30 seconds
            driver.get(url)
            
            # Click on the first element to save the first page
            element1 = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="LeftNavLinks"]/li[8]')))
            element1.click()
            
            time.sleep(15)
            html1 = driver.page_source
            with open(f'C:\\Users\\U6060174\\OneDrive - Clarivate Analytics\\Desktop\\Files\\Jupyter Notebook Work\\Pages\\{name}.html', 'w', encoding='utf-8') as f:
                f.write(html1)
                time.sleep(5)  # add a 5 seconds delay before clicking the second element
            
            # Click on the second element to save the second page
            element2 = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="link_app-data-assignments"]')))
            element2.click()
            
            time.sleep(15)
            html2 = driver.page_source
            with open(f'C:\\Users\\U6060174\\OneDrive - Clarivate Analytics\\Desktop\\Files\\Jupyter Notebook Work\\Pages\\{assignments_name}.html', 'w', encoding='utf-8') as f:
                f.write(html2)
                time.sleep(5)  # add a 5 seconds delay before moving to the next URL
            break  # break out of the retry loop if successful
        except (TimeoutException, WebDriverException) as e:
            logging.error(f"Error occurred while saving the page for {url}: {e}")
            print(f"Error occurred while saving the page for {url}: {e}")
            time.sleep(10)  # add a 10 seconds delay before retrying
    else:  # executed if the retry loop completes without success
        logging.error(f"Failed to save the page after {retry_count} retries for {url}")
        print(f"Failed to save the page after {retry_count} retries for {url}")
    
driver.quit()


In [7]:
# Set the path to the Excel file containing the URLs manual funtionality
url_file_path = r'C:\Users\U6060174\OneDrive - Clarivate Analytics\Desktop\Files\Jupyter Notebook Work\Excel File\url_parse.xlsx'

# Read the Excel file into a pandas dataframe
df_urls = pd.read_excel(url_file_path)

# Create an empty dataframe to store the parsed data
df_data = pd.DataFrame(columns=['Name', 'Data', 'Status'])

# Loop through the URLs in the dataframe and parse the HTML
for i, row in df_urls.iterrows():
    name = row['{Name}']
    file_path = f'C:\\Users\\U6060174\\OneDrive - Clarivate Analytics\\Desktop\\Files\\Jupyter Notebook Work\\Pages\\{name}.html'
    status = 'Success' # set default status as success
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            html = f.read()

        soup = BeautifulSoup(html, 'html.parser')

        text = soup.get_text(separator='\n')
        text = text.replace('<div>', '<div> ')
        text = text.replace('\n', ' \n')
    except Exception as e:
        print(f"Error occurred while processing the file for {name}: {e}")
        text = ''
        status = 'Error' # set status as error if there's an exception
    
    df_data = pd.concat([df_data, pd.DataFrame({'Name': name, 'Data': text, 'Status': status}, index=[0])])

# Write the parsed data to an Excel file
output_file_path = r'C:\Users\U6060174\OneDrive - Clarivate Analytics\Desktop\Files\Jupyter Notebook Work\Excel File\scraped_data_assignments.xlsx'
df_data.to_excel(output_file_path, index=False)
